In [ ]:
import cv2
import numpy as np
import moviepy.editor as mp

def process_frame(frame, roi_width_ratio=9/10):
    # Define the region of interest (ROI) for the specified width ratio
    height, width = frame.shape[:2]
    roi_width = int(width * roi_width_ratio)
    roi_height = height * 4 // 5
    roi = frame[roi_height:, :roi_width]

    # Convert the ROI to grayscale
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to detect text regions
    _, thresh = cv2.threshold(gray_roi, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)


    # Morphological operations to remove noise and connect text components
    kernel = np.ones((3, 3), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find contours in the thresholded ROI
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize variables to store bounding boxes
    all_subtitles_boxes = [[]]  # Initialize with an empty list

    
    #****************************************************************************************
    
    for i, contour in enumerate(contours):
        # Get the bounding box of the contour
        x, y, w, h = cv2.boundingRect(contour)

        # Filter out small contours (noise)
        if w > 5 and h > 5:
            # Check if it's a new line based on vertical distance
            if i > 0:
                _, prev_y, _, _ = cv2.boundingRect(contours[i - 1])
                vertical_distance = y - prev_y
                if vertical_distance > h * 1.5:  # Adjust the threshold as needed
                    # Start a new line
                    all_subtitles_boxes.append([])

            # Update the bounding box coordinates for all subtitles
            subtitle_box = (x, y + roi_height, x + w, y + h + roi_height)
            all_subtitles_boxes[-1].append(subtitle_box)

            # Draw the green bounding box for each word
            cv2.rectangle(frame, (x, y + roi_height), (x + w, y + h + roi_height), (0, 255, 0), thickness=2)

#***********************************************************************************************************************

#***************************************************************************************************************
    # Draw the red bounding box around all subtitles
    for line_boxes in all_subtitles_boxes:
        if line_boxes:
            min_x = min(box[0] for box in line_boxes)
            min_y = min(box[1] for box in line_boxes)
            max_x = max(box[2] for box in line_boxes)
            max_y = max(box[3] for box in line_boxes)
            cv2.rectangle(frame, (min_x, min_y), (max_x, max_y), (0, 0, 255), thickness=2)
            
            
        return all_subtitles_boxes
#************************************************************************************************************
def process_video(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create VideoWriter for output video with AVI format
    out = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))

    # Create a list to store frames for audio processing
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Process the frame (ignoring far right)
        all_subtitles_boxes = process_frame(frame)

        # Display the frame with bounding boxes
        cv2.imshow('Video Subtitles Detector', frame)
        out.write(frame)

        # Append the frame to the list for audio processing
        frames.append(frame)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Process audio
    audio_clip = mp.AudioFileClip(video_path)
    audio = audio_clip.set_fps(fps)
    video_clip = mp.ImageSequenceClip(frames, fps=fps)

    # Combine video and audio
    final_clip = video_clip.set_audio(audio)
    final_clip.write_videofile('output_video_with_audio.avi', codec='libxvid', audio_codec='mp3', fps=fps)
#********************************************************************************************************************************
# Specify the path to the input video
video_path ='F:\Shady\FCAI\Fouth year\image processing\Project\Project Video.mp4'

# Process the video and save the output with sound
process_video(video_path)
#********************************************************************************************************************************